In [2]:
import pandas as pd
import numpy as np
import shutil
import os

In [4]:
dataset = pd.read_csv(f"./data/tdsc/train/labels.csv", dtype={'Case_id': int, 'Label': str, 'Data_path': str, 'Mask_path': str}).set_index('case_id')

In [6]:
print(dataset.head())

        label           data_path           mask_path
case_id                                              
0           M  DATA\DATA_000.nrrd  MASK\MASK_000.nrrd
1           B  DATA\DATA_001.nrrd  MASK\MASK_001.nrrd
2           M  DATA\DATA_002.nrrd  MASK\MASK_002.nrrd
3           M  DATA\DATA_003.nrrd  MASK\MASK_003.nrrd
4           B  DATA\DATA_004.nrrd  MASK\MASK_004.nrrd


In [9]:
malignants = dataset[(dataset == 'M').any(axis=1)]
benigns = dataset[(dataset == 'B').any(axis=1)]
print(malignants.head())
print(benigns.head())

        label           data_path           mask_path
case_id                                              
0           M  DATA\DATA_000.nrrd  MASK\MASK_000.nrrd
2           M  DATA\DATA_002.nrrd  MASK\MASK_002.nrrd
3           M  DATA\DATA_003.nrrd  MASK\MASK_003.nrrd
6           M  DATA\DATA_006.nrrd  MASK\MASK_006.nrrd
8           M  DATA\DATA_008.nrrd  MASK\MASK_008.nrrd
        label           data_path           mask_path
case_id                                              
1           B  DATA\DATA_001.nrrd  MASK\MASK_001.nrrd
4           B  DATA\DATA_004.nrrd  MASK\MASK_004.nrrd
5           B  DATA\DATA_005.nrrd  MASK\MASK_005.nrrd
7           B  DATA\DATA_007.nrrd  MASK\MASK_007.nrrd
11          B  DATA\DATA_011.nrrd  MASK\MASK_011.nrrd


In [10]:
print(len(malignants), len(benigns))

58 42


In [28]:
validation_mal = np.random.randint(0,58,12)
validation_beg = np.random.randint(0,42,8)

print(validation_mal)
print(validation_beg)

[24  8 34  2 36 35  0 25 18 48 50 51]
[16  2 35 11  0  7 18 32]


In [33]:
for i, data in enumerate(malignants.iterrows()):
    idx, data = data
    mask_path = data['mask_path'].replace('\\', '/')
    data_path = data['data_path'].replace('\\', '/')
    label = data['label']
    if i in validation_mal:
        shutil.copy(f"./data/tdsc/train/{mask_path}", f"./data/tdsc/validation/mask/m/{idx}.nrrd")
        shutil.copy(f"./data/tdsc/train/{data_path}", f"./data/tdsc/validation/data/m/{idx}.nrrd")
    else:
        shutil.copy(f"./data/tdsc/train/{mask_path}", f"./data/tdsc/train/mask/m/{idx}.nrrd")
        shutil.copy(f"./data/tdsc/train/{data_path}", f"./data/tdsc/train/data/m/{idx}.nrrd")
        
    
for i, data in enumerate(benigns.iterrows()):
    idx, data = data
    mask_path = data['mask_path'].replace('\\', '/')
    data_path = data['data_path'].replace('\\', '/')
    label = data['label']
    if i in validation_mal:
        shutil.copy(f"./data/tdsc/train/{mask_path}", f"./data/tdsc/validation/mask/b/{idx}.nrrd")
        shutil.copy(f"./data/tdsc/train/{data_path}", f"./data/tdsc/validation/data/b/{idx}.nrrd")
    else:
        shutil.copy(f"./data/tdsc/train/{mask_path}", f"./data/tdsc/train/mask/b/{idx}.nrrd")
        shutil.copy(f"./data/tdsc/train/{data_path}", f"./data/tdsc/train/data/b/{idx}.nrrd")
